# load required modules

In [1]:
import sys, os, pygmt
from pathlib import Path
sys.path.insert(0, '/home/581/da1339/AFIM/src/AFIM/src')
from sea_ice_processor   import SeaIceProcessor
from sea_ice_plotter     import SeaIcePlotter
from grounded_iceberg_processor import GroundedIcebergProcessor
import numpy             as np
import pandas            as pd
import xarray            as xr
from datetime            import timedelta, date, datetime
import matplotlib.pyplot as plt
import matplotlib.dates  as mdates

# grounded icebergs

In [ ]:
GI_proc = GroundedIcebergProcessor()
GI_proc.process_raw_grounded_iceberg_database()
GI_proc.modify_landmask_with_grounded_icebergs(p_min=0.1, p_max=0.9, scaling_exponent=.66)

In [ ]:
GI_proc.write_modified_landmask_and_counts_to_disk(write=True, overwrite=True)

In [ ]:
SP = SeaIcePlotter(ice_type='fi', sim_name='baseline', show_figs=Falsoe)
SP.plot_landmask_differences_per_region()

# process for fast ice

In [ ]:
dt0_str  = "1993-08-01"
dtN_str  = "1993-11-30"
sim_name = 'baseline'
SI_proc  = SeaIceProcessor(sim_name                 = sim_name, 
                           ice_divergence_threshold = 0.01,
                           fast_ice_masks           = ['aice','divu'])
FI_div = SI_proc.process_window(dt0_str    = dt0_str,
                                dtN_str    = dtN_str, 
                                write_zarr = False,
                                ow_zarrs   = True)

In [ ]:
FI_div.FIA

# process sea ice

In [ ]:
SI_proc  = SeaIceProcessor(sim_name = sim_name,
                           sea_ice  = True)
SI_base = SI_proc.process_window(dt0_str   = dt0_str,
                                dtN_str    = dtN_str, 
                                write_zarr = True,
                                ow_zarrs   = False)

In [ ]:
dt0_str  = "1994-01-01"
dtN_str  = "1994-12-31"
sim_name = 'baseline'
plotter = SeaIcePlotter(sim_name  = sim_name,
                        ice_type  = 'FI',
                        plot_type = 'regional',
                        var_name  = 'FIP',
                        dt0_str   = dt0_str,
                        dtN_str   = dtN_str,
                        show_fig  = True,
                        save_fig  = False,
                        overwrite = True)
kwargs = {'fill_color': 'black' , 'sq_size_GI' : 0.1 }
plotter.plot_map( **kwargs )

In [ ]:
sim_name = 'baseline'
ds = xr.open_mfdataset(f"/g/data/gv90/da1339/afim_output/{sim_name}/FI/fast_ice_199*.zarr", engine='zarr')
time = pd.to_datetime(ds['t_dim'].values)
fia = ds['FIA']
fia_obs = ds['FIA_OBS'].sel(sector='circumpolar')
# --- Model: Monthly cycle by year ---
fia_df = pd.DataFrame({'FIA': fia.values}, index=time)
fia_df['Year'] = fia_df.index.year
fia_df['Month'] = fia_df.index.month
monthly_fia = fia_df.groupby(['Year', 'Month']).mean().reset_index()
monthly_cycle = monthly_fia.pivot(index='Month', columns='Year', values='FIA')
# --- Obs: Monthly climatology (mean over available time) ---
# If FIA_OBS is already time-averaged, we assume it has a 12-month length
obs_df = fia_obs.to_dataframe().dropna().reset_index()
# Try parsing month info from the time (or fallback if no datetime index)
if 't_dim' in obs_df.columns and np.issubdtype(obs_df['t_dim'].dtype, np.datetime64):
    obs_df['Month'] = obs_df['t_dim'].dt.month
elif 'Month' not in obs_df.columns:
    obs_df['Month'] = range(1, len(obs_df)+1)
monthly_obs = obs_df.groupby('Month')['FIA_OBS'].mean()

# --- Plot ---
plt.figure(figsize=(12, 6))
plt.style.use('ggplot')

# Plot each year from model
for year in monthly_cycle.columns:
    plt.plot(monthly_cycle.index, monthly_cycle[year], label=f"{year}", linewidth=2)

# Plot observed climatology
plt.plot(monthly_obs.index, monthly_obs.values, label='Observed FIA (AF2020db)', 
         color='black', linestyle='--', linewidth=3)

plt.title("Monthly Fast Ice Area Cycle (1993–1999)", fontsize=16)
plt.xlabel("Month", fontsize=14)
plt.ylabel(f"Fast Ice Area ({fia.attrs.get('units', 'unknown')})", fontsize=14)
plt.xticks(ticks=range(1,13), labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(loc='upper right', fontsize=10, title='Year')
plt.tight_layout()
plt.show()
plt.savefig(f"/g/data/gv90/da1339/GRAPHICAL/timeseries/{sim_name}_FIA_grouped_by_month.png")

In [ ]:
# sim_name = 'baseline'
ds = xr.open_mfdataset(f"/g/data/gv90/da1339/afim_output/{sim_name}/FI/fast_ice_199*.zarr", engine='zarr')
var_name = 'FIV'
# Convert time to datetime index
time = pd.to_datetime(ds['t_dim'].values)
plot_da = ds[var_name]/1e12
# --- Model: Monthly cycle by year ---
plot_df = pd.DataFrame({var_name: plot_da.values}, index=time)
plot_df['Year'] = plot_df.index.year
plot_df['Month'] = plot_df.index.month
monthly_plot = plot_df.groupby(['Year', 'Month']).mean().reset_index()
monthly_cycle = monthly_plot.pivot(index='Month', columns='Year', values=var_name)
plt.figure(figsize=(12, 6))
plt.style.use('ggplot')
for year in monthly_cycle.columns:
    plt.plot(monthly_cycle.index, monthly_cycle[year], label=f"{year}", linewidth=2)
plt.title("Monthly Fast Ice Thickness Cycle (1993–1999)", fontsize=16)
plt.xlabel("Month", fontsize=14)
plt.ylabel(f"Fast Ice Volume ({fia.attrs.get('units', 'unknown')})", fontsize=14)
plt.xticks(ticks=range(1,13), labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(loc='upper right', fontsize=10, title='Year')
plt.tight_layout()
plt.show()
plt.savefig(f"/g/data/gv90/da1339/GRAPHICAL/timeseries/{sim_name}_{var_name}_grouped_by_month.png")

In [ ]:
SI_proc  = SeaIceProcessor(sim_name            = sim_name, 
                           ice_speed_threshold = 1e-3)
FI_hi_spd = SI_proc.process_window(dt0_str    = dt0_str,
                                    dtN_str    = dtN_str, 
                                    write_zarr = False,
                                    ow_zarrs   = True)

In [ ]:
plt.figure(figsize=(12, 6))
plt.style.use('ggplot') 
plt.plot(FI_hi_spd['t_dim'], FI_hi_spd['FIA']    , label=f'{sim_name}-simulation' , linewidth=2, color='tab:blue')
plt.plot(FI_hi_spd['t_dim'], FI_hi_spd['FIA_OBS'], label='Observed FIA (AF2020db)', linewidth=2, linestyle='--', color='tab:orange')
plt.title("Fast Ice Area with simulation results masked with ice speed of <1 mm/s", fontsize=16)
plt.xlabel("Date", fontsize=14)
plt.ylabel(f"Fast Ice Area ({FI_lo_spd.FIA.attrs.get('units', 'unknown')})", fontsize=14)
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gcf().autofmt_xdate()
plt.grid(True, linestyle='--', alpha=0.5)
plt.legend(loc='upper left', fontsize=12)
plt.tight_layout()
plt.show()

# plot 1D analysis

In [ ]:
dt0_str  = "1993-01-01"
dtN_str  = "1993-12-31"
sim_name = 'ktens-max'
plotter  = SeaIcePlotter(sim_name, dt0_str, dtN_str, show_figs=True, ice_type='FI')
plotter.plot_timeseries()

In [ ]:
#plotter.plot_timeseries(var_names=["PI_STRESS"])
plotter.plot_timeseries_compare(
    comparison_name = "tensile-stars",
    sim_names       = ["baseline", "ktens-max", "Pstar-max", "Cstar-min"],
    var_name        = "PIA",
    label_dict      = {"baseline"  : "tensile = 0.2, Pstar = 2.75e4, Cstar = 20",
                       "ktens-max" : "tensile = 0.3, Pstar = 2.75e4, Cstar = 20",
                       "Pstar-max" : "tensile = 0.2, Pstar = 5e4   , Cstar = 20",
                       "Cstar-min" : "tensile = 0.2, Pstar = 2.75e4, Cstar = 10"})

# plot 2D analysis (requires existing FI/PI dataset)

In [ ]:
dt0_str  = "1993-01-01"
dtN_str  = "1999-12-31"
sim_names = ["baseline", "ustar-low", "Cstar-max", "Cstar-min", "Pstar-max", "Pstar-min",
             "ktens-nil", "ktens-min", "ktens-max", "gi-max", "gi-min",
             "Rothrock", "elps-pack", "elps-fast", "elps-max", "elps-def", "elps-mid" ]
for sim_name in sim_names:
    SI_plot = SeaIcePlotter(sim_name, dt0_str, dtN_str, show_figs=False, overwrite=True, single_figure=False)
    SI_plot.prepare_data_for_plotting(var_name='FIP')
    SI_plot.plot_regions()

# generate PBS jobs for analysis

In [ ]:
import numpy as np
from matplotlib.colors import LinearSegmentedColormap, to_rgb

# Step 1: Define your ColorBrewer hex codes
hex_colors = [
    "#ffffff",  # added white start
    "#ffffd9", "#edf8b1", "#c7e9b4", "#7fcdbb",
    "#41b6c4", "#1d91c0", "#225ea8", "#253494", "#081d58"
]

# Step 2: Convert hex to RGB (0–1 range)
rgb_colors = [to_rgb(hex) for hex in hex_colors]

# Step 3: Create a matplotlib colormap and sample 256 values
cmap = LinearSegmentedColormap.from_list("custom_ygnbu", rgb_colors, N=256)
sampled_rgb = cmap(np.linspace(0, 1, 256))[:, :3]  # strip alpha channel

# Step 4: Generate CPT lines
cpt_lines = []
for i in range(255):
    val1 = i
    val2 = i + 1
    r1, g1, b1 = (sampled_rgb[i] * 255).astype(int)
    r2, g2, b2 = (sampled_rgb[i+1] * 255).astype(int)
    cpt_lines.append(f"{val1} {r1} {g1} {b1} {val2} {r2} {g2} {b2}")

# Add background (B), foreground (F), and NaN (N) colors
cpt_lines.append("B 255 255 255")
cpt_lines.append("F 0 0 0")
cpt_lines.append("N 128 128 128")

# Save to a .cpt file
with open("/g/data/gv90/da1339/GRAPHICAL/CPTs/AF2020_YlGnBu.cpt", "w") as f:
    f.write("\n".join(cpt_lines))


In [ ]:
from pathlib import Path
from datetime import datetime, timedelta

type_of_sea_ice = 'sea_ice'     #'pack_ice' or 'sea_ice'
turn_on_overwrite = False

if turn_on_overwrite:
    overwrite = '--overwrite'
else:
    overwrite = ''

if type_of_sea_ice=='fast_ice':
    abbreviation = 'FI'
    switch       = ''
elif type_of_sea_ice=='pack_ice':
    abbreviation = 'PI'
    switch       = '--pack_ice'
elif type_of_sea_ice=='sea_ice':
    abbreviation = 'SI'
    switch       = '--sea_ice'

sim_names = ["Cstar-max", "Cstar-min", "Pstar-max", "Pstar-min", "Rothrock","baseline", "oras-sfc-ctl",
             "elps-def", "elps-fast", "elps-max", "elps-mid", "elps-pack", "gi-max",
             "gi-min", "ktens-max", "ktens-min", "ktens-nil", "ustar-low", "van_achter" ]

# Base time: 5 minutes from now
base_time = datetime.now() + timedelta(minutes=5)

pbs_template = """#!/bin/bash
#PBS -N {abbrev}-proc-{name}
#PBS -P gv90
#PBS -l walltime=24:00:00
#PBS -q normalbw
#PBS -l mem=64GB
#PBS -l ncpus=28
#PBS -l storage=gdata/cj50+gdata/jk72+scratch/jk72+gdata/ik11+gdata/hh5+gdata/rt52+gdata/gb6+gdata/gv90
#PBS -M daniel.atwater@utas.edu.au
#PBS -a {start_time}

module purge
module use /g/data/hh5/public/modules
module load conda/analysis3

cd ~/AFIM/src/AFIM/scripts
python3 ./compute_sea_ice.py {name} {switch} {overwrite}
"""

output_dir = Path(f"/home/581/da1339/AFIM/src/sh/analysis/{type_of_sea_ice}")
output_dir.mkdir(exist_ok=True)

for i, sim in enumerate(sim_names):
    start_dt = base_time + timedelta(minutes=5 * i)
    start_str = start_dt.strftime("%y%m%d%H%M")  # YYMMDDhhmm format
    script_content = pbs_template.format(name=sim, start_time=start_str, switch=switch, overwrite=overwrite, abbrev=abbreviation)
    script_path = output_dir / f"{abbreviation}-proc_{sim}.pbs"
    print(f"writing file: {script_path}")
    with open(script_path, "w") as f:
        f.write(script_content)

print(f"✓ Generated {len(sim_names)} staggered PBS scripts in: {output_dir.resolve()}")
